# Google Trends 關鍵詞爬蟲
https://trends.google.com.tw/trends/trendingsearches/daily?geo=TW

## 載入套件

In [ ]:
import re
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup
import datetime

## 抓取一天的關鍵詞

In [ ]:
# 如果連接掛掉可以自己把ed中的參數改成今天的日期~
url = 'https://trends.google.com.tw/trends/api/dailytrends?hl=zh-TW&tz=-480&ed=20200119&geo=TW&ns=15'
resp = requests.get(url)
resp.text

')]}\',\n{"default":{"trendingSearchesDays":[]}}'

In [ ]:
pd.DataFrame(json.loads(re.sub(r'\)\]\}\',\n', '', resp.text))['default']['trendingSearchesDays'][0]['trendingSearches'])

IndexError: ignored

## 抓取近一月的熱門關鍵詞

In [ ]:
enddt = datetime.datetime.today()
startdt = enddt - datetime.timedelta(days=29)
print(enddt)
print(startdt)

2020-02-18 15:37:27.608796
2020-01-20 15:37:27.608796


In [ ]:
df = []
for i in pd.date_range(start=datetime.datetime.strftime(startdt,'%Y%m%d'), end=datetime.datetime.strftime(enddt,'%Y%m%d'), freq='1D'):
    url = 'https://trends.google.com.tw/trends/api/dailytrends?hl=zh-TW&tz=-480&ed={}&geo=TW&ns=15'.format(datetime.datetime.strftime(i, '%Y%m%d'))
    print(url)
    resp = requests.get(url)
    ndf=[]
    ndf = pd.DataFrame(json.loads(re.sub(r'\)\]\}\',\n', '', resp.text))['default']['trendingSearchesDays'][0]['trendingSearches'])
    ndf['date'] = datetime.datetime.strftime(i, '%Y-%m-%d')
    df.append(ndf)
df = pd.concat(df, ignore_index=True)
df['title'] = df['title'].apply(lambda x: x['query'])
df

https://trends.google.com.tw/trends/api/dailytrends?hl=zh-TW&tz=-480&ed=20200120&geo=TW&ns=15
https://trends.google.com.tw/trends/api/dailytrends?hl=zh-TW&tz=-480&ed=20200121&geo=TW&ns=15
https://trends.google.com.tw/trends/api/dailytrends?hl=zh-TW&tz=-480&ed=20200122&geo=TW&ns=15
https://trends.google.com.tw/trends/api/dailytrends?hl=zh-TW&tz=-480&ed=20200123&geo=TW&ns=15
https://trends.google.com.tw/trends/api/dailytrends?hl=zh-TW&tz=-480&ed=20200124&geo=TW&ns=15
https://trends.google.com.tw/trends/api/dailytrends?hl=zh-TW&tz=-480&ed=20200125&geo=TW&ns=15
https://trends.google.com.tw/trends/api/dailytrends?hl=zh-TW&tz=-480&ed=20200126&geo=TW&ns=15
https://trends.google.com.tw/trends/api/dailytrends?hl=zh-TW&tz=-480&ed=20200127&geo=TW&ns=15
https://trends.google.com.tw/trends/api/dailytrends?hl=zh-TW&tz=-480&ed=20200128&geo=TW&ns=15
https://trends.google.com.tw/trends/api/dailytrends?hl=zh-TW&tz=-480&ed=20200129&geo=TW&ns=15
https://trends.google.com.tw/trends/api/dailytrends?hl=zh-TW

,title,formattedTraffic,relatedQueries,image,articles,shareUrl,date
0,武漢肺炎,20萬+,[],{'newsUrl': 'https://www.bbc.com/zhongwen/trad...,"[{'title': '武漢新型病毒有變異可能確診及死亡人數攀升', 'timeAgo': ...",https://trends.google.com.tw/trends/trendingse...,2020-01-20
1,武漢,5萬+,[],{'newsUrl': 'https://udn.com/news/story/7314/4...,"[{'title': '還有10團<b>武漢</b>團要來台玩林佳龍：協調取消', 'tim...",https://trends.google.com.tw/trends/trendingse...,2020-01-20
2,SARS,5萬+,[],{'newsUrl': 'https://technews.tw/2020/01/22/wu...,[{'title': '德國病毒權威：武漢肺炎疫情嚴重程度不如SARS | TechNews...,https://trends.google.com.tw/trends/trendingse...,2020-01-20
3,N95口罩,2萬+,"[{'query': 'n95', 'exploreLink': '/trends/expl...",{'newsUrl': 'https://health.tvbs.com.tw/medica...,[{'title': '防冠狀病毒戴N95口罩未必最好，這一款正確用就有效 健康2.0'...,https://trends.google.com.tw/trends/trendingse...,2020-01-20
4,武漢肺炎病徵,2萬+,[],{'newsUrl': 'https://www.commonhealth.com.tw/a...,[{'title': '與武漢肺炎病毒博鬥22天痊癒「幸好我年輕，全身器官都在幫我」 - 康...,https://trends.google.com.tw/trends/trendingse...,2020-01-20
...,...,...,...,...,...,...,...
594,李明亮,2000+,[],{'newsUrl': 'https://www.setn.com/news.aspx?Ne...,"[{'title': '疫程才剛開始？李明亮說這事刻不容緩', 'timeAgo': '1 ...",https://trends.google.com.tw/trends/trendingse...,2020-02-18
595,湖南獨立宣言,2000+,[],{'newsUrl': 'https://www.setn.com/News.aspx?Ne...,"[{'title': '網傳《湖南獨立宣言》：脫離中共暴政', 'timeAgo': '14...",https://trends.google.com.tw/trends/trendingse...,2020-02-18
596,張昭雄,2000+,[],{'newsUrl': 'https://tw.news.yahoo.com/%E6%B0%...,"[{'title': '張昭雄見證台灣棒球崛起引進日職資訊', 'timeAgo': '1 ...",https://trends.google.com.tw/trends/trendingse...,2020-02-18
597,洪堯,2000+,[],{'newsUrl': 'https://www.hk01.com/%E5%8D%B3%E6...,"[{'title': '【延禧攻略】吳謹言洪堯戀情再次斷正？ 網友爆兩人三亞同宿', 'ti...",https://trends.google.com.tw/trends/trendingse...,2020-02-18


## 保存資料

### 串接 Google Drive 空間

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### 保存資料
- 資料會保存在 Google Drive 上 Colab Notebooks/WebCrawler/GoogleTrends/ 的資料夾空間中

In [ ]:
import os
path = '/content/drive/My Drive/Colab Notebooks/WebCrawler/GoogleTrends/'
# 如果不存在這個資料夾就新建這個資料夾
if not os.path.exists(path):
    os.makedirs(path)
df.to_excel(path + 'GoogleTrends_keywords.xlsx')